In [ ]:
#reading text files from URL
!wget -r --no-parent "http://eng1.mu.edu.tr/~tugba/Pubchem/aid_divided_files"

In [ ]:
#returns the names of the files in the directory data as a list
import os
list_of_files = os.listdir("/content/eng1.mu.edu.tr/~tugba/Pubchem/aid_divided_files")
lines=[]
i=0
list_of_txt=[]
list_of_name=[]
for file in list_of_files:
     list_of_name.append(file)
     file="/content/eng1.mu.edu.tr/~tugba/Pubchem/aid_divided_files/"+file
     f=open(file, "r")
     i+=1
     
     list_of_txt.append(f.readlines()[0])
 
 
 
#df_list_of_name = pd.DataFrame(data=list_of_name)
print(len(list_of_name))
#df_list_of_name.to_csv(r'/content/list_of_name.csv')

2416


In [ ]:
#producing stopwords for data
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')


#creating stopwords for data
def create_stopwords(text_file,name_of_text):
  
  stop_w={}
  stopword_list=[]
  num_of_file=len(name_of_text)
  z=0;
  for i in text_file:
      for j in nltk.word_tokenize(i):
          if  j.lower() not in stopwords.words("english"):
              if j not in stop_w  :

                  stop_w[j]=[name_of_text[z]]
              else:
                if name_of_text[z] not in  stop_w[j]:
                     stop_w[j]+=[name_of_text[z]]
        
      z+=1
  print(len(stop_w))
  for name in stop_w:

    if (len(stop_w[name])/num_of_file) > 0.3:
       stopword_list.append(name.lower())
  
  return stopword_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#creating stopwords
s=create_stopwords(list_of_txt,list_of_name)
print(len(s))



24405
45


In [ ]:
#Tokenizer, Tfidf vectorizer and kmeans cluster
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from numpy import unique
from numpy import  where
from matplotlib import pyplot
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
nltk.download('punkt')
nltk.download('wordnet')
import string
from nltk.corpus import stopwords
nltk.download('stopwords')




trans_table = {ord(c): None for c in string.punctuation + string.digits}
nltk.download("wordnet", "../nltk_data/")

#s=create_stopwords(list_of_txt,list_of_name)
new_stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",
           "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
           'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',
           'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
           'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
           'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
           'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above',
           'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then',
           'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
           'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
           'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
           "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
           "haven't", 'isn', "isn't", 'ma', 'mightn',"phd",
           "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]


#adding stopwords to english stopwords
new_stopwords+=s


#Tokenizer fonction
def tokenize(text):
        text = re.sub(r'\s+\d+', '', text)
        
        #tokens = [word for word in nltk.word_tokenize(text.translate(trans_table)) if len(word) > 2 ]
        tokens = [word.lower() for word in nltk.word_tokenize(text.translate(trans_table)) if len(word) > 3 ] 
        stems = [lemmatizer.lemmatize(item) for item in tokens ]
        stems =[  item for item in stems if item not in new_stopwords]
        if("phd"  in stems):
          print("soor")
        return stems


#TfIdVectorazier fitting
vectorizer = TfidfVectorizer(stop_words='english',tokenizer=tokenize)
X = vectorizer.fit_transform(list_of_txt)

#converting feactures to dataframe
output=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

#print(vectorizer.get_feature_names())

#Clustering Kmeans
kmeans = KMeans(n_clusters=6).fit(X)




output['max_tfidf_value'] = output.max(axis=1)
output["word_of_max_tfidf"]=output.idxmax(axis=1)
output["aa_names"]=list_of_name
output["Cluster"]=kmeans.labels_


#print (output)
#converting output to csv(to use in web)
#output.to_csv(r'/content/output.csv')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to ../nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.



In [ ]:
#convert mutiltble dimensions to three dimentions
from sklearn.decomposition import PCA

# First: for every document we get its corresponding cluster
clusters = kmeans.predict(X)

# We train the PCA on the dense version of the tf-idf. 
pca = PCA(n_components=3)
two_dim = pca.fit_transform(X.todense())

scatter_x = two_dim[:, 0] # first principle component
scatter_y = two_dim[:, 1] # second principle component
scatter_z = two_dim[:, 2]  # third principle component

In [ ]:
#print most 10  words occur in clusters

def takeWordsInCluster(wordCount,kValue):
      order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
      terms = vectorizer.get_feature_names()
      words_dic={}
      for i in range(kValue):
          
          name="Cluster :" +str(i)
          words_list=[]
          for ind in order_centroids[i, :wordCount]:
             words_list.append(terms[ind])
              
          words_dic[name]=words_list

      df = pd.DataFrame(data=words_dic)
      return df

most_words_list=takeWordsInCluster(10,6)
most_words_list.to_csv(r'/content/most_words_list.csv')

In [ ]:
#ploting 2d 
import numpy as np
import plotly.express as px


output['Cluster'] = output['Cluster'].astype(str)  #cluster changeing string

#Creating new Datframe for clustering
d = {'pca_0':scatter_x, 'pca_1': scatter_y,'pca_2': scatter_z, "cluster":output['Cluster'],"name":list_of_name,"max_tfidf_value":output['max_tfidf_value'] ,"word_of_max_tfidf":output["word_of_max_tfidf"]}
df = pd.DataFrame(data=d)


fig = px.scatter(df, x="pca_0", y="pca_1",size_max=20,
                 size="max_tfidf_value",hover_data=["name",'cluster',"word_of_max_tfidf","max_tfidf_value"], color="cluster", color_discrete_sequence=px.colors.qualitative.Safe, template="simple_white")



new_frame={'names':output["aa_names"],"cluster":output['Cluster']}
new_frame = pd.DataFrame(data=new_frame)
new_frame.to_csv(r'/content/file_name_cluster_name.csv')

fig.show()

#Converting and exporting csv file to use in web 
df.to_csv(r'/content/cluster.csv')

In [ ]:
#ploting 3d
fig = px.scatter_3d(df, x="pca_0", y="pca_1",z="pca_2",
              color='cluster', template="simple_white")
fig.show()
